
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #0


Algorithme glouton

### 1. Préparation des données



In [1]:
# modules importés
# lecture de fichiers csv et dict colonnes
import csv as csv
# nettoyage des caractères via expression regex
import re as re
# mesure du temps passé -> time spend over complexity
import time
# mesure de l'occupation mémoire space complexity
from sys import argv


In [2]:
# constants
FILE = "data/p7-20-shares.csv" 
FIELDNAMES = ['name', 'cost', 'profit']
STEP = 100 
BUDGET = 500 * STEP


In [3]:
# check if file name was passed as parm to the script
if __name__ == '__main__':
    if len(argv) == 2:
        FILE = sys.argv[1]

In [4]:
 
def fn_timer(function):
    """ starts before & stops after the run of the function, a time counter"""        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        elapsed = (t1-t0)/1000000000
        print(f"Total time running {function.__name__}: {str(elapsed)}s seconds")
        return result
    return function_timer

In [5]:
# strips a string from its weird caracters
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

In [6]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
file_name = FILE
try:
    with open(file_name, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=',', doublequote=False)
        # skip the header
        next(csv_reader)
        compteur_ligne = 0
        for idx, line in enumerate(csv_reader):
            clean_data = True

            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False

            if line[FIELDNAMES[1]] != "":
                cout = int(STEP * float(clean_char(line[FIELDNAMES[1]])))
                if cout < 0 :
                    print(f" line {idx} had neg cost data; dropped.")
                    cout = 0
                    clean_data = False                    
                if cout == 0 :
                    print(f" line {idx} had null cost data ; could have been a gift but management decision: dropped.")
                    cout = 0
                    clean_data = False                                        
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
                
            if line[FIELDNAMES[2]] != "":
                gain_percent = int(STEP * float(clean_char(line[FIELDNAMES[2]])))
            else:
                print(f" line {idx} had missing profit percentage; dropped.")
                clean_data = False
            if gain_percent <= 0:
                # TODO: check if to keep or not in any case comment ; as negativ can't be optimum
                print(f"** line {idx} had negative profit percentage ; accepted but pls check. **")
                print('      ',idx,line)    
            if clean_data:
                action_dict[cle] = (cout, cout*gain_percent/STEP)
                compteur_ligne += 1
            else:
                print('      ',idx,line)    
        print("nombre d'actions retenues: ", compteur_ligne)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom dans le répertoire data {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")      


nombre d'actions retenues:  20


In [7]:
#action_dict
                

## 2. Algorithme Glouton

[Algorithme glouton — Wikipédia](https://fr.wikipedia.org/wiki/Algorithme_glouton)

Un **algorithme glouton** (_greedy algorithm_ en anglais, parfois appelé aussi algorithme gourmand, ou goulu) est un [algorithme](https://fr.wikipedia.org/wiki/Algorithmique "Algorithmique") qui suit le principe de faire, étape par étape, un choix optimum local, dans l'espoir d'obtenir un résultat optimum global. Par exemple, dans le problème du rendu de monnaie (donner une somme avec le moins possible de pièces), l'algorithme consistant à répéter le choix de la pièce de plus grande valeur qui ne dépasse pas la somme restante est un algorithme glouton.
Dans le système de pièces (1, 3, 4), l'algorithme glouton n'est pas optimal, comme le montre l'exemple simple suivant. Il donne pour 6 : 4+1+1, alors que 3+3 est optimal. 
Mais optimal par rapport à quelle contrainte? à priori le nombre pièce utilisé. Si c'était avoir moins de pièce dans son portemonnaie, la 1ère solution était "optimale".

In [11]:
@fn_timer
def algo_glouton(dictio: dict[str,tuple],budget: int) -> (int, list):
    """ Tri par densité de profit càd que les actions au meilleur rapport profit/cout
        sont les premières dans l'ordre
    """
    action_trie = dict(sorted(dictio.items(), key=lambda x:x[1][1]/x[1][0], reverse=True))
    budget_left = budget
    action_selected: list[str] = []
    for cle,valeur in action_trie.items():
        if budget_left - valeur[0] > 0:
            action_selected.append(cle)
            budget_left -= valeur[0]
    return (budget_left,action_selected)

In [12]:
# V2 : executons la fonction sur les données précédentes
reste_budget, action_pf = algo_glouton(action_dict,BUDGET)

Total time running algo_glouton: 2.27e-05s seconds


In [16]:
print('At cost of ',(BUDGET-reste_budget)/STEP, ' ', action_pf, ' actions brought a profit of ',round(sum(list(map(lambda x: action_dict[x][1],action_pf)))/(STEP*STEP),2))

At cost of  498.0   ['Action-10', 'Action-6', 'Action-13', 'Action-19', 'Action-4', 'Action-20', 'Action-5', 'Action-11', 'Action-18', 'Action-17', 'Action-16', 'Action-14']  actions brought a profit of  97.48
